<a href="https://colab.research.google.com/github/redeian/Deep_Intent_classification_Keras/blob/master/Intent_classification_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
import nltk
import re
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout
from keras.callbacks import ModelCheckpoint


Using TensorFlow backend.


In [0]:
def load_dataset(filename):
  df = pd.read_csv(filename, encoding = "latin1", names = ["Sentence", "Intent"])
  print(df.head())
  intent = df["Intent"]
  unique_intent = list(set(intent))
  sentences = list(df["Sentence"])
  
  return (intent, unique_intent, sentences)
  


In [0]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [0]:
file_path = '/content/Dataset.csv'


In [5]:
intent, unique_intent, sentences = load_dataset(file_path)

                Sentence          Intent
0       Need help pleese  commonQ.assist
1              Need help  commonQ.assist
2       I need some info  commonQ.assist
3      Will you help me?  commonQ.assist
4  What else can you do?  commonQ.assist


In [6]:
print(sentences[:5])

['Need help pleese', 'Need help', 'I need some info', 'Will you help me?', 'What else can you do?']


In [7]:
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
#define stemmer
stemmer = LancasterStemmer()

In [0]:
def cleaning(sentences):
  words = []
  for s in sentences:
    clean = re.sub(r'[^ a-z A-Z 0-9]', " ", s)
    w = word_tokenize(clean)
    #stemming
    words.append([i.lower() for i in w])
    
  return words  

In [10]:
cleaned_words = cleaning(sentences)
print(len(cleaned_words))
print(cleaned_words[:2])  
  


1113
[['need', 'help', 'pleese'], ['need', 'help']]


In [0]:
def create_tokenizer(words, filters = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'):
  token = Tokenizer(filters = filters)
  token.fit_on_texts(words)
  return token

In [0]:
def max_length(words):
  return(len(max(words, key = len)))
  

In [13]:
word_tokenizer = create_tokenizer(cleaned_words)
vocab_size = len(word_tokenizer.word_index) + 1
max_length = max_length(cleaned_words)

print("Vocab Size = %d and Maximum length = %d" % (vocab_size, max_length))

Vocab Size = 492 and Maximum length = 28


In [0]:
def encoding_doc(token, words):
  return(token.texts_to_sequences(words))

In [0]:
encoded_doc = encoding_doc(word_tokenizer, cleaned_words)

In [0]:
def padding_doc(encoded_doc, max_length):
  return(pad_sequences(encoded_doc, maxlen = max_length, padding = "post"))

In [0]:
padded_doc = padding_doc(encoded_doc, max_length)

In [18]:
padded_doc[:5]

array([[ 25,  77, 332,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [ 25,  77,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  1,  25, 198, 181,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [ 51,  10,  77,  16,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  8, 268,   4,  10,  30,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]], dtype=int32)

In [19]:
print("Shape of padded docs = ",padded_doc.shape)

Shape of padded docs =  (1113, 28)


In [0]:
#tokenizer with filter changed
output_tokenizer = create_tokenizer(unique_intent, filters = '!"#$%&()*+,-/:;<=>?@[\]^`{|}~')


In [21]:
output_tokenizer.word_index

{'commonq.assist': 1,
 'commonq.bot': 16,
 'commonq.how': 12,
 'commonq.just_details': 15,
 'commonq.name': 11,
 'commonq.not_giving': 9,
 'commonq.query': 10,
 'commonq.wait': 13,
 'contact.contact': 8,
 'faq.aadhaar_missing': 4,
 'faq.address_proof': 3,
 'faq.application_process': 18,
 'faq.apply_register': 5,
 'faq.approval_time': 7,
 'faq.bad_service': 2,
 'faq.banking_option_missing': 21,
 'faq.biz_category_missing': 14,
 'faq.biz_new': 17,
 'faq.biz_simpler': 20,
 'faq.borrow_limit': 19,
 'faq.borrow_use': 6}

In [0]:
encoded_output = encoding_doc(output_tokenizer, intent)

In [0]:
encoded_output = np.array(encoded_output).reshape(len(encoded_output), 1)

In [24]:
encoded_output.shape

(1113, 1)

In [0]:
def one_hot(encode):
  o = OneHotEncoder(sparse = False)
  return(o.fit_transform(encode))

In [26]:
output_one_hot = one_hot(encoded_output)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [27]:
output_one_hot.shape

(1113, 21)

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
train_X, val_X, train_Y, val_Y = train_test_split(padded_doc, output_one_hot, shuffle = True, test_size = 0.2)


In [30]:
print("Shape of train_X = %s and train_Y = %s" % (train_X.shape, train_Y.shape))
print("Shape of val_X = %s and val_Y = %s" % (val_X.shape, val_Y.shape))

Shape of train_X = (890, 28) and train_Y = (890, 21)
Shape of val_X = (223, 28) and val_Y = (223, 21)


In [0]:
def create_model(vocab_size, max_length):
  model = Sequential()
  model.add(Embedding(vocab_size, 128, input_length = max_length, trainable = False))
  model.add(Bidirectional(LSTM(128)))
#   model.add(LSTM(128))
  model.add(Dense(32, activation = "relu"))
  model.add(Dropout(0.5))
  model.add(Dense(21, activation = "softmax"))
  
  return model

In [32]:
model = create_model(vocab_size, max_length)

model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 28, 128)           62976     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               263168    
_________________________________________________________________
dense_1 (Dense)              (None, 32)                8224      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 21)                693       
Total params: 335,061
Trainable params: 272,085
Non-trainable params: 62,976
__________

In [33]:
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

hist = model.fit(train_X, train_Y, epochs = 100, batch_size = 32, validation_data = (val_X, val_Y), callbacks = [checkpoint])

Instructions for updating:
Use tf.cast instead.
Train on 890 samples, validate on 223 samples
Epoch 1/100
890/890 [==============================] - 4s 5ms/step - loss: 2.9160 - acc: 0.1157 - val_loss: 2.8026 - val_acc: 0.2108

Epoch 00001: val_loss improved from inf to 2.80257, saving model to model.h5
Epoch 2/100
890/890 [==============================] - 3s 3ms/step - loss: 2.8147 - acc: 0.1348 - val_loss: 2.7294 - val_acc: 0.2108

Epoch 00002: val_loss improved from 2.80257 to 2.72941, saving model to model.h5
Epoch 3/100
890/890 [==============================] - 3s 3ms/step - loss: 2.7647 - acc: 0.1629 - val_loss: 2.5958 - val_acc: 0.1973

Epoch 00003: val_loss improved from 2.72941 to 2.59578, saving model to model.h5
Epoch 4/100
890/890 [==============================] - 3s 3ms/step - loss: 2.6566 - acc: 0.1708 - val_loss: 2.6275 - val_acc: 0.2511

Epoch 00004: val_loss did not improve from 2.59578
Epoch 5/100
890/890 [==============================] - 3s 3ms/step - loss: 2.628

In [0]:
 model = load_model("model.h5")

In [0]:
def predictions(text):
  clean = re.sub(r'[^ a-z A-Z 0-9]', " ", text)
  test_word = word_tokenize(clean)
  test_word = [w.lower() for w in test_word]
  test_ls = word_tokenizer.texts_to_sequences(test_word)
  print(test_word)
  #Check for unknown words
  if [] in test_ls:
    test_ls = list(filter(None, test_ls))
    
  test_ls = np.array(test_ls).reshape(1, len(test_ls))
 
  x = padding_doc(test_ls, max_length)
  
  pred = model.predict_proba(x)
  
  
  return pred


  

In [0]:
def get_final_output(pred, classes):
  predictions = pred[0]
 
  classes = np.array(classes)
  ids = np.argsort(-predictions)
  classes = classes[ids]
  predictions = -np.sort(-predictions)
 
  for i in range(pred.shape[1]):
    print("%s has confidence = %s" % (classes[i], (predictions[i])))



In [37]:
text = "suck"
pred = predictions(text)
get_final_output(pred, unique_intent)

['suck']
faq.bad_service has confidence = 0.09750116
commonQ.query has confidence = 0.095663205
commonQ.how has confidence = 0.09041219
faq.apply_register has confidence = 0.08783795
commonQ.bot has confidence = 0.08211021
faq.aadhaar_missing has confidence = 0.07156516
commonQ.wait has confidence = 0.06425594
commonQ.name has confidence = 0.058494292
faq.biz_new has confidence = 0.0528298
faq.borrow_limit has confidence = 0.05233835
commonQ.not_giving has confidence = 0.04119842
commonQ.assist has confidence = 0.036291532
contact.contact has confidence = 0.028934397
commonQ.just_details has confidence = 0.022363873
faq.banking_option_missing has confidence = 0.021732463
faq.biz_category_missing has confidence = 0.021567991
faq.application_process has confidence = 0.01962978
faq.approval_time has confidence = 0.018830411
faq.biz_simpler has confidence = 0.015840832
faq.address_proof has confidence = 0.010443282
faq.borrow_use has confidence = 0.010158773
